<a href="https://colab.research.google.com/github/ArtyA23/Maker_Portfolio_Colab_Code/blob/main/Training_emotional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
import sys
import subprocess
import torch
from transformers import TrainingArguments, Trainer, AutoModelForCausalLM, AutoTokenizer
import os
from transformers.trainer_utils import get_last_checkpoint

# 1. Check and upgrade transformers if needed
def ensure_transformers_version(min_version="4.4.0"):
    try:
        import transformers
        from packaging import version
        if version.parse(transformers.__version__) < version.parse(min_version):
            print(f"[INFO] Upgrading transformers from {transformers.__version__} to latest...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "transformers"])
            print("[INFO] Restart your script after upgrade finishes.")
            sys.exit(0)
    except ImportError:
        print("[INFO] Installing transformers...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers"])
        print("[INFO] Restart your script after install finishes.")
        sys.exit(0)

ensure_transformers_version()

#  3. Load preprocessed dataset (saves soooo much time!)
from datasets import load_from_disk
dataset = load_from_disk("/content/drive/MyDrive/Attreya_AI_Colab_Project/Attreya_Maker_Portfolio-rasA/Data-ONLY_FOR_VIEWING/processed_data-emotional")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

# Add pad token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    tokenizer.pad_token = tokenizer.eos_token  # or use '[PAD]' if you prefer

def tokenize_function(examples):
    # assuming you have combined text in "text" column
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    # causal LM needs labels
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True)

split_dataset = tokenized_dataset["train"].train_test_split(test_size=0.1)
train_data = split_dataset["train"]
test_data = split_dataset["test"]

tokenized_dataset.save_to_disk("processed_data")

print(dataset["train"][0]["text"])

#  3.5. Declare device (to prevent errors :)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#  4. Load the model onto GPU explicitly
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

if torch.cuda.is_available():
    model = model.to("cuda")
#  5. Training configurations
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Attreya_AI_Colab_Project/Attreya_Maker_Portfolio-rasA/Data-ONLY_FOR_VIEWING/processed_data-emotional",
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=8,
    weight_decay=0.01,
    logging_dir="logs",
    logging_steps=500,
    save_strategy="steps",
    save_steps=40000,
    eval_strategy="steps",
    save_total_limit=2,
    fp16=True,
    push_to_hub=False,
    dataloader_num_workers=0,
    dataloader_pin_memory=True,
     overwrite_output_dir=False,
    load_best_model_at_end=False,
)

last_checkpoint = None
if os.path.isdir(training_args.output_dir):
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is not None:
        print(f"Found last checkpoint: {last_checkpoint}. Trainer will resume from it.")

model.gradient_checkpointing_enable()
#  6. Data collator
def data_collator(features):
    return {
        "input_ids": torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(f["input_ids"]) for f in features],
            batch_first=True,
            padding_value=tokenizer.pad_token_id
        ),
        "attention_mask": torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(f["attention_mask"]) for f in features],
            batch_first=True,
            padding_value=0
        ),
        "labels": torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(f["input_ids"]) for f in features],
            batch_first=True,
            padding_value=-100
        )
    }

#  7. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

try:
    # pass path string if found (or None). Trainer accepts bool/path.
    trainer.train(resume_from_checkpoint=last_checkpoint)
except Exception as e:
    print("Training interrupted with exception:", e)
    print("Saving current model and tokenizer to disk to avoid losing progress...")
    # best-effort save
    trainer.save_model(training_args.output_dir)              # saves model/config
    tokenizer.save_pretrained(training_args.output_dir)       # saves tokenizer
    raise  # re-raise so you still see the original error/traceback

# optional final save (trainer.save_model is equivalent to model.save_pretrained)
trainer.save_model(training_args.output_dir)
tokenizer.save_pretrained(training_args.output_dir)

#  8. Manual evaluation after each epoch
num_epochs = int(training_args.num_train_epochs)
for epoch in range(num_epochs):
    print(f"\n===== Epoch {epoch+1}/{num_epochs} =====")
    trainer.train(resume_from_checkpoint=False)
    metrics = trainer.evaluate()
    print("f Evaluation metrics after epoch {epoch+1}: {metrics}")

#  9. Save model & tokenizer
model.save_pretrained("/content/drive/MyDrive/Attreya_AI_Colab_Project/Attreya_Maker_Portfolio-rasA/AI_Model-ONLY_FOR_VIEWING/trained_model")
tokenizer.save_pretrained("/content/drive/MyDrive/Attreya_AI_Colab_Project/Attreya_Maker_Portfolio-rasA/AI_Model-ONLY_FOR_VIEWING/trained_model")


FileNotFoundError: Directory /content/drive/MyDrive/Attreya_Maker_Portfolio-rasA/Data-ONLY_FOR_VIEWING/processed_data-emotional not found